In [1]:
!pip3 install hdbscan 
!pip3 install bertopic

In [2]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
from tqdm import tqdm 
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/express/imshow_utils.py:24: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/

In [4]:
tweets_c = pd.read_csv('/Users/lydiafield/Desktop/Semestre 1/Digital Methods/bookbans-nolinks.csv')

In [79]:
tweets_c["State"].value_counts()

State
New York            1662
Florida             1315
California          1288
Texas                834
Washington, D.C.     740
Illinois             528
Pennsylvania         389
North Carolina       362
Tennessee            309
Washington           292
Virginia             276
Massachusetts        263
Iowa                 242
Missouri             238
New Jersey           234
Michigan             209
Indiana              193
Ohio                 180
Minnesota            175
Georgia              157
Arizona              150
Kansas               146
Maryland             141
Colorado             136
Oregon               104
Arkansas             103
Utah                  96
Wisconsin             94
Connecticut           87
Rhode Island          83
Oklahoma              82
Kentucky              80
South Carolina        80
Louisiana             72
Nevada                62
Alabama               59
Mississippi           50
North Dakota          50
Nebraska              47
Idaho              

In [22]:
def str_check(df,list,us_state,auth_list):
    df1 = tweets_c[tweets_c['Hit Sentence'].str.contains('|'.join(list))]
    df2 = tweets_c[tweets_c['State'].str.contains(us_state,na=False)]
    df3 = tweets_c[tweets_c['Influencer'].str.contains('|'.join(auth_list))]
    df = pd.concat([df1,df2,df3])
    return df.reset_index()

Florida Tweets

In [43]:
fl_tweets = pd.DataFrame().reindex_like(tweets_c)
fl_tweets = fl_tweets.drop(fl_tweets.index[1:20000])

In [69]:
fl_tweets= str_check(fl_tweets,['DeSantis', 'desantis', 'Desantis', 'florida', 'fl ', 'FL ', 'pushaw', 'Pushaw', 'DESANTIS', 'Florida', 
                                'floridian', 'floridians', 'Floridian', 'Floridians', '@RonDeSantis', '@GovRonDeSantis', '@DeSantisWarRoom',
                                '@RonDeSantisFL', 'HB 1069', 'HB1069', 'Clay County', 'Martin County', 'Manatee County', 'Flagler', 
                                'Osceola County', 'Escambia County', 'Lake County', 'Orlando', 'Seminole County', 'Tallahassee', 
                                'Indian River County', 'Volusia County', 'F.L.', 'F.L', 'Fla', ' fla ', 'hb 1069', 'hb1069'],
                     'Florida',
                     ['@RonDeSantis', '@GovRonDeSantis', '@DeSantisWarRoom', '@RonDeSantisFL'])

In [70]:
len(fl_tweets)

5501

In [265]:
sum(fl_tweets['Engagement'])

6148246

California Tweets

In [47]:
ca_tweets = pd.DataFrame().reindex_like(tweets_c)
ca_tweets = ca_tweets.drop(ca_tweets.index[1:20000])

In [164]:
ca_tweets= str_check(ca_tweets,['Newsom', 'newsom', 'NEWSOM', 'California', 'CA ', 'california', 'cali', 'CAgovernor',
                                ' ca ', ' Ca ', 'C.A.', 'C.A', 'cali', 'Cali', 'AB 1078', 'AB1078', 'ab1078', 'ab 1078',
                               'Thurmond', 'Temecula'],
                     'California',
                     ['@GavinNewsom', '@CAgovernor', '@GovPressOffice', '@JenSiebelNewsom'])

In [165]:
len(ca_tweets)

1913

In [266]:
sum(ca_tweets['Engagement'])

3816166

New York Tweets

In [75]:
ny_tweets = pd.DataFrame().reindex_like(tweets_c)
ny_tweets = ny_tweets.drop(ny_tweets.index[1:20000])

In [90]:
ny_tweets= str_check(ny_tweets,['New York', 'new york', ' NY', ' ny', 'N.Y.', 'N.Y', ' Ny ', 'NYC', 'nyc', 'Albany', 'albany',
                                'Hochul', 'hochul', 'Eric Adams', 'long island', 'Long Island', 'Island Trees', 
                               'Hudson Valley', 'Westchester', 'Brooklyn', 'Queens', 'Manhattan', 'Wappingers', 'NYCLU'],
                     'New York',
                     ['@GovKathyHocul', '@KathyHochul', '@NYCMayor', '@ericadamsfornyc', 'NYCMayorsOffice'])

In [91]:
len(ny_tweets)

2026

In [267]:
sum(ny_tweets['Engagement'])

2606770

Texas Tweets

In [89]:
tx_tweets = pd.DataFrame().reindex_like(tweets_c)
tx_tweets = tx_tweets.drop(tx_tweets.index[1:20000])

In [209]:
tx_tweets= str_check(tx_tweets,['Texas', 'texas', 'TX', ' Tex', ' tx ', 'Tx ', 'Abbott', 'abbott', 'Ted Cruz', 'Cruz'
                               'Houston', 'Dallas', 'San Antonio', 'El Paso', 'Austin', 'Midland County', 'Katy', 
                                'Beaumont', 'Friendswood', 'HB 3979', 'HB3979', 'abbott'],
                     'Texas',
                     ['@GovAbbott', '@GregAbbott_TX', '@TXGregAbbott', '@texasgov', 'SenTedCruz', '@tedcruz'])

In [210]:
len(tx_tweets)

1641

In [268]:
sum(tx_tweets['Engagement'])

1343758

Illinois Tweets

In [103]:
il_tweets = pd.DataFrame().reindex_like(tweets_c)
il_tweets = il_tweets.drop(il_tweets.index[1:20000])

In [207]:
il_tweets= str_check(il_tweets,['Illinois', 'illinois', ' IL', ' il ', 'I.L.', 'I.L ', ' Il', 'Pritzker', 'HB 2789', 
                                'HB2789', 'hb2789', 'hb 2789', 'Chicago', 'chicago', 'Giannoulias', 'pritzker'],
                     'Illinois',
                     ['@GovPritzker', '@JBPritzker', '@ILGovernorPress'])

In [208]:
len(il_tweets)

1205

In [269]:
sum(il_tweets['Engagement'])

1564113

Missouri Tweets

In [111]:
mo_tweets = pd.DataFrame().reindex_like(tweets_c)
mo_tweets = mo_tweets.drop(mo_tweets.index[1:20000])

In [205]:
mo_tweets= str_check(mo_tweets,['Missouri', 'missouri', 'MO ', 'M.O.', 'M.O', ' mo ', 'St. Louis', 'Saint Louis', 
                                'Parson', 'Missourian', 'parson'],
                     'Missouri', 
                     ['@GovParsonMO', '@mikeparson', '@MoGov', '@CoMissourian'])

In [206]:
len(mo_tweets)

514

In [270]:
sum(mo_tweets['Engagement'])

473462

Utah Tweets

In [118]:
ut_tweets = pd.DataFrame().reindex_like(tweets_c)
ut_tweets = ut_tweets.drop(ut_tweets.index[1:20000])

In [203]:
ut_tweets= str_check(ut_tweets,['Utah', 'utah', 'U.T.', 'Salt Lake City', 'SLC', 'slc', 'Salt Lake', 'Alpine School', 
                                ' UT ', 'Davis School', 'Mormon', 'mormons', 'mormon', 'Latter-Day', 'Provo', 'Cox', 'cox'],
                     'Utah', 
                     ['@GovCox', '@SpencerJCox'])

In [204]:
len(ut_tweets)

397

In [271]:
sum(ut_tweets['Engagement'])

280354

Pennsylvania Tweets

In [138]:
pa_tweets = pd.DataFrame().reindex_like(tweets_c)
pa_tweets = pa_tweets.drop(pa_tweets.index[1:20000])

In [201]:
pa_tweets= str_check(pa_tweets,['Pennsylvania', 'pennsylvania', 'Penn.', 'penn', 'Pittsburgh', 'Philly', 'Philadelphia',
                               'philly', 'phila.', 'Phila.', 'Central Bucks', 'P.A.', ' PA ', ' pa ', 'P.A', 'Harrisburg', 
                               'Oxford Area', 'Cappelletti', 'Friel', 'Lancaster County', 'Bucks County', 'York County', 
                               'shapiro', 'Shapiro'],
                     'Pennsylvania', 
                     ['@GovernorShapiro', '@JoshShapiroPA', '@PennsylvaniaGov', '@SenCappelletti', '@amandaforpa'])

In [202]:
len(pa_tweets)

790

In [272]:
sum(pa_tweets['Engagement'])

665259

Blue States:
NY
CA
IL
WA
MA

Red States:
PA
UT
TX
FL
MO

Washington Tweets

In [166]:
wa_tweets = pd.DataFrame().reindex_like(tweets_c)
wa_tweets = wa_tweets.drop(wa_tweets.index[1:20000])

In [222]:
wa_tweets= str_check(wa_tweets,['Washington state', 'Washington ', ' WA ', 'W.A.', 'W.A', ' wa ', 'Tacoma', 'Olympia', 
                               'Seattle', 'Inslee', 'inslee'],
                     'Washington', 
                     ['@GovInslee', '@JayInslee', '@WAStateGov'])

wa_tweets = wa_tweets.rename(columns = {'Hit Sentence':'Tweet'})

searchfor = ['Washington Post', 'Washington, D.C.', 'Washington, DC', 'Harold Washington']
wa_tweets = wa_tweets[~wa_tweets.Tweet.str.contains('|'.join(searchfor))]

In [223]:
len(wa_tweets)

1120

In [273]:
sum(wa_tweets['Engagement'])

1256302

Massachussets Tweets

In [173]:
ma_tweets = pd.DataFrame().reindex_like(tweets_c)
ma_tweets = ma_tweets.drop(ma_tweets.index[1:20000])

In [175]:
ma_tweets= str_check(ma_tweets,['Massachussetts', 'massachusetts', ' MA ', 'M.A.', 'M.A', ' ma ', 'Boston', 'boston', 
                               'Mass.', 'mass.', 'MassGovernor', 'Healey', 'healey'],
                     'Massachussetts', 
                     ['@MassGovernor', '@maura_healey', '@MassGov'])

In [176]:
len(ma_tweets)

384

In [274]:
sum(ma_tweets['Engagement'])

464314

Save to CSVs

In [215]:
fl_tweets.to_csv('fl_tweet.csv', index=False)
ca_tweets.to_csv('ca_tweet.csv', index=False)
ny_tweets.to_csv('ny_tweet.csv', index=False)
tx_tweets.to_csv('tx_tweet.csv', index=False)
ut_tweets.to_csv('ut_tweet.csv', index=False)
pa_tweets.to_csv('pa_tweet.csv', index=False)
il_tweets.to_csv('il_tweet.csv', index=False)
wa_tweets.to_csv('wa_tweet.csv', index=False)
ma_tweets.to_csv('ma_tweet.csv', index=False)
mo_tweets.to_csv('mo_tweet.csv', index=False)

Concat Blue & Red States

In [251]:
red_states = pd.concat([fl_tweets, tx_tweets, ut_tweets, pa_tweets, mo_tweets])
blue_states = pd.concat([ca_tweets, ny_tweets, il_tweets, wa_tweets, ma_tweets])

In [252]:
red_states['state'] = 'red'
blue_states['state'] = 'blue'

In [253]:
red_blue = pd.concat([red_states,blue_states])

In [255]:
#shuffle dataframe before de-duping
red_blue = red_blue.sample(frac = 1)

In [256]:
len(red_blue)

15491

In [257]:
#de-dupe
red_blue = red_blue.drop_duplicates(subset=['URL'])

In [258]:
len(red_blue)

11270

In [259]:
#re-separate red and blue state dataframes
red_states = red_blue[red_blue.state == 'red'].copy()
blue_states = red_blue[red_blue.state == 'blue'].copy()

In [260]:
len(red_states)

6328

In [261]:
len(blue_states)

4942

In [262]:
red_states.to_csv('red_states.csv', index=False)
blue_states.to_csv('blue_states.csv', index=False)

In [263]:
sum(red_states['Engagement'])

5883267

In [264]:
sum(blue_states['Engagement'])

7217916